In [1]:
import glob
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd
from datetime import datetime, timedelta
import calendar
from collections import Counter
import requests
from multiprocessing import Pool

## Functions

In [4]:
def convert_date(date):
    # Remove special characters
    date = re.sub('[^a-z0-9]', " ", date)
    try:
        # ASSUMING dd MMM
        # Extract day
        day_start = re.search('^\d+', date).start()
        day_end = re.search('^\d+', date).end()
        day = date[day_start:day_end]
        if len(day) == 1:
            day = "0"+day
        # Extract month
        date_wo_day = date[day_end:].strip()
        month_start = re.search('^\w+', date_wo_day).start()
        month_end = re.search('^\w+', date_wo_day).end()
        month = date_wo_day[month_start:month_end]
        if "jan" in str(month):
            month = "01"
        if "feb" in str(month):
            month = "02"
        if "mar" in str(month):
            month = "03"
        if "apr" in str(month):
            month = "04"
        if "may" in str(month):
            month = "05"
        if "jun" in str(month):
            month = "06"
        if "jul" in str(month):
            month = "07"
        if "aug" in str(month):
            month = "08"
        if "sep" in str(month):
            month = "09"
        if "oct" in str(month):
            month = "10"
        if "nov" in str(month):
            month = "11"
        if "dec" in str(month):
            month = "12"

        full_date = day+"."+month+".2019"
        if len(full_date) == 10:
            return datetime.strptime(full_date, '%d.%m.%Y').date()
        else:
            return "nil"
    except:
        return "nil"

In [12]:
def convert_time(time, post_time):
    return_time = "nil"
    time_original = str(re.sub(r'[.:]', '', time)).strip()
    time_original = str(re.sub(r'^([a-zA-Z]+ ){1,}', '', time_original)).strip()
    time_original = str(re.sub(r'\d+\-', '', time_original)).strip()
    
    # ASSUME NUM PM
    if bool(re.search(r'^\d+ ?pm', time_original)):
        time = time_original.replace("pm", "").strip()
        time = str(time)
        # If 1,2,3,4,5,6,7,8,9
        if len(str(time)) == 1:
            time = int(time)
            if time == 1:
                return_time = "13:00"
            if time == 2:
                return_time = "14:00"
            if time == 3:
                return_time = "15:00"
            if time == 4:
                return_time = "16:00"
            if time == 5:
                return_time = "17:00"
            if time == 6:
                return_time = "18:00"
            if time == 7:
                return_time = "19:00"
            if time == 8:
                return_time = "20:00"
            if time == 9:
                return_time = "21:00"
        # If 10,11,12
        if len(str(time)) == 2:
            time = int(time)
            if time == 10:
                return_time = "22:00"
            if time == 11:
                return_time = "23:00"
            if time == 12:
                return_time = "12:00"
        # If single hour (eg. 1) and minutes (eg. 15)
        if len(str(time)) == 3:
            hour = int(time[0])
            if hour == 1:
                return_time = "13:"+time[1:]
            if hour == 2:
                return_time = "14:"+time[1:]
            if hour == 3:
                return_time = "15:"+time[1:]
            if hour == 4:
                return_time = "16:"+time[1:]
            if hour == 5:
                return_time = "17:"+time[1:]
            if hour == 6:
                return_time = "18:"+time[1:]
            if hour == 7:
                return_time = "19:"+time[1:]
            if hour == 8:
                return_time = "20:"+time[1:]
            if hour == 9:
                return_time = "21:"+time[1:]
        # If double hour (eg. 11) and minutes (eg. 30)
        if len(str(time)) == 4:
            hour = int(time[0:2])
            if hour == 10:
                return_time = "22:"+time[2:]
            if hour == 11:
                return_time = "23:"+time[2:]
            if hour == 12:
                return_time = "12:"+time[2:]
    
    # ASSUME NUM AM
    if bool(re.search(r'^\d+ ?am', time_original)):
        time = time_original.replace("am", "").strip()
        time = str(time)
        # If 1,2,3,4,5,6,7,8,9
        if len(str(time)) == 1:
            time = int(time)
            if time == 1:
                return_time = "01:00"
            if time == 2:
                return_time = "02:00"
            if time == 3:
                return_time = "03:00"
            if time == 4:
                return_time = "04:00"
            if time == 5:
                return_time = "05:00"
            if time == 6:
                return_time = "06:00"
            if time == 7:
                return_time = "07:00"
            if time == 8:
                return_time = "08:00"
            if time == 9:
                return_time = "09:00"
        # If 10,11,12
        if len(str(time)) == 2:
            time = int(time)
            if time == 10:
                return_time = "10:00"
            if time == 11:
                return_time = "11:00"
            if time == 12:
                return_time = "00:00"
        # If single hour (eg. 1) and minutes (eg. 15)
        if len(str(time)) == 3:
            hour = int(time[0])
            if hour == 1:
                return_time = "01:"+time[1:]
            if hour == 2:
                return_time = "02:"+time[1:]
            if hour == 3:
                return_time = "03:"+time[1:]
            if hour == 4:
                return_time = "04:"+time[1:]
            if hour == 5:
                return_time = "05:"+time[1:]
            if hour == 6:
                return_time = "06:"+time[1:]
            if hour == 7:
                return_time = "07:"+time[1:]
            if hour == 8:
                return_time = "08:"+time[1:]
            if hour == 9:
                return_time = "09:"+time[1:]
        # If double hour (eg. 11) and minutes (eg. 30)
        if len(str(time)) == 4:
            hour = int(time[0:2])
            if hour == 10:
                return_time = "10:"+time[2:]
            if hour == 11:
                return_time = "11:"+time[2:]
            if hour == 12:
                return_time = "00:"+time[2:]
    
    # ASSUME NO PM OR AM
    if bool(re.search(r'^\d+$', time_original)):
        time = str(time_original).strip()
        # If single digit
        if len(time) == 1:
            time = int(time)
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if time == 1:
                    return_time = "13:00"
                if time == 2:
                    return_time = "14:00"
                if time == 3:
                    return_time = "15:00"
                if time == 4:
                    return_time = "16:00"
                if time == 5:
                    return_time = "17:00"
                if time == 6:
                    return_time = "18:00"
                if time == 7:
                    return_time = "19:00"
                if time == 8:
                    return_time = "20:00"
                if time == 9:
                    return_time = "21:00"
            else:
                if time == 1:
                    return_time = "01:00"
                if time == 2:
                    return_time = "02:00"
                if time == 3:
                    return_time = "03:00"
                if time == 4:
                    return_time = "04:00"
                if time == 5:
                    return_time = "05:00"
                if time == 6:
                    return_time = "06:00"
                if time == 7:
                    return_time = "07:00"
                if time == 8:
                    return_time = "08:00"
                if time == 9:
                    return_time = "09:00"
        # If double digits
        if len(str(time)) == 2:
            time = int(time)
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if time == 10:
                    return_time = "22:00"
                if time == 11:
                    return_time = "23:00"
                if time == 12:
                    return_time = "12:00"
            else:
                if time == 10:
                    return_time = "10:00"
                if time == 11:
                    return_time = "11:00"
                if time == 12:
                    return_time = "00:00"    
        # If three digtis
        if len(str(time)) == 3:
            hour = int(str(time)[0])
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if hour == 1:
                    return_time = "13:"+time[1:]
                if hour == 2:
                    return_time = "14:"+time[1:]
                if hour == 3:
                    return_time = "15:"+time[1:]
                if hour == 4:
                    return_time = "16:"+time[1:]
                if hour == 5:
                    return_time = "17:"+time[1:]
                if hour == 6:
                    return_time = "18:"+time[1:]
                if hour == 7:
                    return_time = "19:"+time[1:]
                if hour == 8:
                    return_time = "20:"+time[1:]
                if hour == 9:
                    return_time = "21:"+time[1:]
            else:
                if hour == 1:
                    return_time = "01:"+time[1:]
                if hour == 2:
                    return_time = "02:"+time[1:]
                if hour == 3:
                    return_time = "03:"+time[1:]
                if hour == 4:
                    return_time = "04:"+time[1:]
                if hour == 5:
                    return_time = "05:"+time[1:]
                if hour == 6:
                    return_time = "06:"+time[1:]
                if hour == 7:
                    return_time = "07:"+time[1:]
                if hour == 8:
                    return_time = "08:"+time[1:]
                if hour == 9:
                    return_time = "09:"+time[1:]
        # If four digtis
        if len(str(time)) == 4:
            hour = int(str(time)[0:2])
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if hour >= 12:
                    return_time = str(hour)+":"+str(time)[2:]
                if hour == 10:
                    return_time = "22:"+time[2:]
                if hour == 11:
                    return_time = "23:"+time[2:]
                if hour == 0:
                    return_time = "00:"+time[2:]
            else:
                if hour <= 12:
                    if len(str(hour)) == 1:
                        return_time = "0"+str(hour)+":"+str(time)[2:]
                    else:
                        return_time = str(hour)+":"+str(time)[2:]
                if hour == 0:
                    return_time = "00:"+time[2:]
                if hour == 10:
                    return_time = "10:"+time[2:]
                if hour == 11:
                    return_time = "11:"+time[2:]
                if hour == 12:
                    return_time = "00:"+time[2:]
        
    return return_time

## Process Data

### Read Files

In [6]:
html_files_old = glob.glob("./telegram_chat_export_beginning_19sep/*.html")
print("Number of files: ", len(html_files_old))
print(html_files_old[:10])
print()

html_files_new = glob.glob("./telegram_chat_export_13oct_10dec/*.html")
print("Number of files: ", len(html_files_new))
print(html_files_new[:10])

Number of files:  257
['./telegram_chat_export_beginning_19sep\\messages.html', './telegram_chat_export_beginning_19sep\\messages10.html', './telegram_chat_export_beginning_19sep\\messages100.html', './telegram_chat_export_beginning_19sep\\messages101.html', './telegram_chat_export_beginning_19sep\\messages102.html', './telegram_chat_export_beginning_19sep\\messages103.html', './telegram_chat_export_beginning_19sep\\messages104.html', './telegram_chat_export_beginning_19sep\\messages105.html', './telegram_chat_export_beginning_19sep\\messages106.html', './telegram_chat_export_beginning_19sep\\messages107.html']

Number of files:  160
['./telegram_chat_export_13oct_10dec\\messages.html', './telegram_chat_export_13oct_10dec\\messages10.html', './telegram_chat_export_13oct_10dec\\messages100.html', './telegram_chat_export_13oct_10dec\\messages101.html', './telegram_chat_export_13oct_10dec\\messages102.html', './telegram_chat_export_13oct_10dec\\messages103.html', './telegram_chat_export_1

In [7]:
html_files = [*html_files_old, *html_files_new]
print("Total html files: ", len(html_files))

Total html files:  417


### Extract Data from Files

In [64]:
messages_list = []
total_messages = 0

for file in tqdm(html_files):    
    f = open(file, "r", encoding='utf-8')
    chat = f.read()
    soup = BeautifulSoup(chat)
    messages = soup.findAll("div", {"class": "message default clearfix"})
    total_messages += len(messages)

    for message in messages:
        # Get user
        user = message.find("div", {"class": "from_name"}).text.replace("\n", "").strip()
        #print(user)
        # Get post date and time
        post_datetime = message.find("div", {"class": "pull_right date details"}).get("title")
        post_date = post_datetime.split(" ")[0]
        post_time = post_datetime.split(" ")[1]
        #print(post_date)
        #print(post_time)
        # GET MESSAGE
        raw_message = message.find("div", {"class": "text"})
        # Extract text from message
        raw_message = str(raw_message).replace("</div>", "").replace('<div class="text">', "").replace("<strong>", "").replace("</strong>", "").replace("<u>", "").replace("</u>", "").strip().lower()
        # Tokenize text
        raw_message_tokens = re.sub(r'(<br/> ?){2,}', '<br/>', raw_message).split("<br/>")

        # Variable flag for actual ride request
        is_request = False
        # Get user type
        if(bool(re.search(r'hitchers? looking', raw_message_tokens[0]))):
            user_type = "hitcher"
            is_request = True
        elif(bool(re.search(r'driver? looking', raw_message_tokens[0]))):
            user_type = "driver"
            is_request = True
        else:
            user_type = "nil"

        # PROCESS MESSAGE CONTENT IF IT IS ACTUALLY A REQUEST
        if(is_request):
            # Blank place holder
            pick_up = "nil"
            drop_off = "nil"
            request_date = "nil"
            request_time = "nil"
            number_of_pax = "nil"
            request_date_normalised = "nil"
            request_time_normalised = "nil"
            day = "nil"
            # ------------------

            # Extract pick up location assuming it is always the second token
            try:
                pick_up_tokens = str(raw_message_tokens[1]).split(":")
            except:
                pass
            try:
                if "pick" in pick_up_tokens[0]:
                    pick_up = pick_up_tokens[1].strip()
            except:
                pick_up_tokens = str(raw_message_tokens[1]).split(" ")
                if "pick" in pick_up_tokens[0]:
                    pick_up = " ".join(pick_up_tokens[1:])
                else:
                    pass

            # Extract drop off location assuming it is always the third token
            try:
                drop_off_tokens = str(raw_message_tokens[2]).split(":")
            except:
                pass
            try:
                if "drop" in drop_off_tokens[0]:
                    drop_off = drop_off_tokens[1].strip()
            except:
                drop_off_tokens = str(raw_message_tokens[2]).split(" ")
                if "drop" in drop_off_tokens[0]:
                    drop_off = " ".join(drop_off_tokens[1:])
                else:
                    pass

            # Extract date assuming it is always the fourth token
            try:
                date_tokens = str(raw_message_tokens[3]).split(":")
            except:
                pass
            try:
                if "date" in date_tokens[0]:
                    request_date = date_tokens[1].strip()
            except:
                date_tokens = str(raw_message_tokens[3]).split(" ")
                if "date" in date_tokens[0]:
                    request_date = " ".join(date_tokens[1:])
                else:
                    pass
            if "today" in request_date or "tdy" in request_date:
                request_date = datetime.strptime(post_date, '%d.%m.%Y').date()
            elif "tmr" in request_date or "tomorrow" in request_date:
                today_date = datetime.strptime(post_date, '%d.%m.%Y').date()
                tomorrow_date = today_date + timedelta(days=1)
                request_date = tomorrow_date
            elif request_date == "nil":
                pass
            else:
                request_date = convert_date(request_date)
            
            # Extract time assuming it is always the fifth token
            try:
                time_raw = str(raw_message_tokens[4])
            except:
                pass
            try:
                time = time_raw.replace("time:", "").replace("time :", "").strip()
                time = re.sub(r'[^a-z0-9.-:]', '', time).strip()
                if "now" in time:
                    request_time = post_time[:5]
                else:
                    request_time = convert_time(str(time), post_time)
            except:
                pass
                
            # Extract number of pax assuming it is always the sixth token
            try:
                if "pax" in raw_message_tokens[5]:
                    raw_pax = str(raw_message_tokens[5]).replace("pax:", "").replace("pax", "").replace("pax :", "").replace("pax : ", "").strip()
                    raw_pax = raw_pax.replace(":", "").strip()
                    raw_pax = re.sub(r'\(.*\)', '', raw_pax)
                    raw_pax = re.sub(r'[^a-z0-9- ]', '', raw_pax)
                    raw_pax = raw_pax.strip()
                    if len(raw_pax) == 1:
                        number_of_pax = raw_pax
                    elif len(raw_pax) == 3:
                        number_of_pax = raw_pax
                    else:
                        if bool(re.search(r'^\d+ - \d+$', raw_pax)):
                            number_of_pax = raw_pax.replace(" ", "")
                        if bool(re.search(r'^\d+ to \d+$', raw_pax)):
                            number_of_pax = raw_pax.replace("to", "-")
                            number_of_pax = number_of_pax.replace(" ", "")
            except:
                pass
            
            # If request_date and time is "nil" format of message might be wrong
            if request_date == "nil" or request_time == "nil":
                # Extract time again
                try:
                    time_raw = re.search(r'time\:.*\<br\/\>', raw_message).group()
                    time = time_raw.split("<br/>")[0]
                    time = time.replace("time:", "").strip()  
                    if "now" in time:
                        request_time = post_time[:5]
                    else:
                        request_time = convert_time(str(time), post_time)
                except:
                    pass
                # Extract date again
                try:
                    date_raw = re.search(r'date\:.*\<br\/\>', raw_message).group()
                    request_date = date_raw.split("<br/>")[0]
                    request_date = date.replace("date:", "").strip()
                    if "today" in request_date or "tdy" in request_date:
                        request_date = datetime.strptime(post_date, '%d.%m.%Y').date()
                    elif "tmr" in request_date or "tomorrow" in request_date:
                        today_date = datetime.strptime(post_date, '%d.%m.%Y').date()
                        tomorrow_date = today_date + timedelta(days=1)
                        request_date = tomorrow_date
                    else:
                        request_date = convert_date(request_date)
                except:
                    if request_time != "nil":
                        request_date = datetime.strptime(post_date, '%d.%m.%Y').date()
                # Extract pax again
                try:
                    if bool(re.search(r'pax:.*\<br\/\>', raw_message)):
                        raw_pax = re.search(r'pax:.*\<br\/\>', lol).group()
                        raw_pax = raw_pax.split("<br/>")[0]
                        raw_pax = raw_pax.replace("pax:", "").strip()
                    if bool(re.search(r'pax:.*$', lol)):
                        raw_pax = re.search(r'pax:.*$', lol).group()
                        raw_pax = raw_pax.replace("pax:", "").strip()
                except:
                    pass
            
            #--------------------NORMALISE DATA---------------------
            
            # Normalise date and time
            try:
                date_time = datetime.strptime(str(request_date)+" "+request_time, '%Y-%m-%d %H:%M')
                approx = round(date_time.minute/30.0) * 30
                date_time = date_time.replace(minute=0)
                date_time += timedelta(seconds=approx*60)
                request_date_normalised = date_time.date()
                request_time_normalised = date_time.strftime('%H:%M')
            except:
                pass
            
            # Add day to data eg. dd-mm-yyyy = wed
            try:
                my_date = datetime.strptime(str(request_date_normalised), '%Y-%m-%d')
                day = calendar.day_name[my_date.weekday()].lower()
            except:
                pass
            
            # Store structured message into dictionary
            message_data = {
                'user' : user,
                'post_date' : post_date,
                'post_time' : post_time,
                'raw_message' : raw_message,
                'user_type' : user_type,
                'pick_up' : pick_up,
                'drop_off' : drop_off,
                'request_date' : request_date,
                'request_date_normalised' : request_date_normalised,
                'request_time' : request_time,
                'request_time_normalised' : request_time_normalised,
                'day' : day,
                'number_of_pax' : number_of_pax
            }
            messages_list.append(message_data)

100%|████████████████████████████████████████████████████████████████████████████████| 417/417 [05:49<00:00,  1.09it/s]


### Stats

In [65]:
print("Total number of messages: \t\t", total_messages)
print("Total number of actual messages: \t", len(messages_list))
print()
print("Percentage of actual messages: \t", round((len(messages_list)/total_messages)*100, 2), "%")

Total number of messages: 		 376598
Total number of actual messages: 	 360464

Percentage of actual messages: 	 95.72 %


### Convert Data to DataFrame

In [68]:
data = pd.DataFrame(messages_list)
data.head()
data.to_csv("./data/sg_hitch_data_raw_updated.csv")

### Filter out Corrupted Data

In [2]:
data = pd.read_csv("./data/sg_hitch_data_raw_updated.csv", index_col=0)

# Filter out corrupted request time
clean_data_rt = data[data.request_time_normalised != "nil"]
print("Removed", len(data)-len(clean_data_rt), "corrupted request time")
# Filter out corrupted request date
clean_data_rd = clean_data_rt[clean_data_rt.request_date_normalised != "nil"]
print("Removed", len(clean_data_rt)-len(clean_data_rd), "corrupted request date")
# Filter out corrupted pick up
clean_data_pu = clean_data_rd[clean_data_rd.pick_up != "nil"]
print("Removed", len(clean_data_rd)-len(clean_data_pu), "corrupted pick up")
# Filter out corrupted drop off
clean_data_do = clean_data_pu[clean_data_pu.drop_off != "nil"]
print("Removed", len(clean_data_pu)-len(clean_data_do), "corrupted drop off")

Removed 18292 corrupted request time
Removed 0 corrupted request date
Removed 318 corrupted pick up
Removed 2341 corrupted drop off


In [3]:
clean_data = clean_data_do
print("Number of requests left: ", len(clean_data))
print()
print("Percentage of non-corrupted data: ", round(len(clean_data)/len(data)*100, 2), "%")

Number of requests left:  339513

Percentage of non-corrupted data:  94.19 %


In [4]:
clean_data.head()

,day,drop_off,number_of_pax,pick_up,post_date,post_time,raw_message,request_date,request_date_normalised,request_time,request_time_normalised,user,user_type
1,tuesday,bedok reservoir,nil,tuas area,26.02.2019,14:55:47,driver looking for hitchers<br/>pick up: tuas ...,2019-02-26,2019-02-26,18:15,18:00,xoxoxo,driver
2,tuesday,off - ntu,nil,up - mountbatten mrt,26.02.2019,15:00:23,hitcher looking for drivers<br/>pick up - moun...,2019-02-26,2019-02-26,17:00,17:00,Amanda,hitcher
3,tuesday,north-east,nil,bedok/tamp/pasir ris/eunos,26.02.2019,15:39:19,driver looking for hitchers<br/>pick up: bedok...,2019-02-26,2019-02-26,16:00,16:00,J,driver
4,tuesday,north-east or east,nil,jurong,26.02.2019,18:12:47,driver looking for hitchers<br/>pick up: juron...,2019-02-26,2019-02-26,18:12,18:00,Colonel Ronald Silver,driver
7,tuesday,suss,nil,jurong west,26.02.2019,23:50:17,hitcher looking for drivers <br/>pick up: juro...,2019-02-26,2019-02-26,07:30,07:30,a. Lee,hitcher


### Normalise Data

In [9]:
# Normalise Location
pap = pd.read_csv("./data/planning_area_population.csv")
planning_areas = pap['name'].tolist()
len(planning_areas)

55

In [5]:
pick_up = clean_data['pick_up'].tolist()
drop_off = clean_data['drop_off'].to_list()

In [6]:
def clean_location(loc):
    clean = re.sub(r'\bamk\b', 'ang mo kio', loc)
    clean = re.sub(r'\bcck\b', 'choa chu kang', clean)
    clean = re.sub(r'\btpy\b', 'toa payoh', clean)
    clean = re.sub(r'\byck\b', 'yio chu kang', clean)
    lean = re.sub(r'^vivo$', 'vivo city', clean)
    clean = re.sub(r'\btamp\b', 'tampines', clean)
    clean = re.sub(r'\bseng kang\b', 'sengkang', clean)
    clean = re.sub(r'\bcq\b', 'clarke quay', clean)
    return clean

In [7]:
pick_up_clean = []
drop_off_clean = []

for pu in pick_up:
    pick_up_clean.append(clean_location(str(pu)))
    
for do in drop_off:
    drop_off_clean.append(clean_location(str(do)))

In [91]:
print(Counter(pick_up_clean).most_common(100))
#print(Counter(drop_off_clean).most_common(20))

[('woodlands', 8149), ('yishun', 7530), ('tampines', 6932), ('sengkang', 6813), ('punggol', 5396), ('hougang', 5243), ('choa chu kang', 4932), ('ang mo kio', 4881), ('pasir ris', 4850), ('jurong west', 3921), ('bedok', 3505), ('jurong', 3247), ('bukit batok', 2898), ('sembawang', 2881), ('bukit panjang', 2737), ('clarke quay', 1951), ('town', 1868), ('toa payoh', 1746), ('serangoon', 1580), ('clementi', 1501), ('bugis', 1382), ('jurong east', 1337), ('liang court', 1323), ('orchard', 1300), ('bishan', 1182), ('ntu', 1144), ('mbs', 1069), ('yew tee', 880), ('tanjong pagar', 856), ('boon lay', 817), ('nus', 795), ('geylang', 737), ('changi airport', 735), ('simei', 732), ('khatib', 724), ('woodland', 715), ('jewel', 705), ('admiralty', 703), ('kovan', 698), ('vivo', 681), ('jurong point', 651), ('holland village', 647), ('tiong bahru', 641), ('yewtee', 620), ('ubi', 604), ('scape', 602), ('vivo city', 597), ('ngee ann poly', 592), ('marsiling', 587), ('west', 584), ('313 somerset', 579),

In [10]:
count = 0
for pu in tqdm(drop_off_clean):
    for pa in planning_areas:
        if pu == pa:
            count += 1

100%|██████████████████████████████████████████████████████████████████████| 339513/339513 [00:01<00:00, 277539.28it/s]


In [11]:
count/len(pick_up_clean)*100

23.787012573892603

In [12]:
# ONEMAP API TO GET ADDRESS
def get_address(location):
    onemap_url = "https://developers.onemap.sg/commonapi/search"
    onemap_params = {'searchVal' : location, 'returnGeom' : "Y", 'getAddrDetails' : "Y"}
    r = requests.get(url = onemap_url, params = onemap_params)
    data = r.json()
    print(data['results'][0]['ADDRESS'])

In [104]:
clean_data.to_csv("./data/sg_hitch_data.csv")

In [ ]:
with Pool(2) as p:
    p.map(get_address, pick_up_clean[:10])